In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pathlib
import matplotlib.pyplot as plt
from PIL import Image

In [57]:
import logging


def init_logging(level=logging.INFO):
    logging.basicConfig(level=level,
                        format='%(asctime)s %(levelname)s: %(message)s [in %(pathname)s %(lineno)s]')

init_logging(level=logging.DEBUG)
logger=logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.debug("debug logging")
logger.info("info logging")

2022-07-16 18:35:26,873 DEBUG: debug logging [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\2382164240.py 11]
2022-07-16 18:35:26,874 INFO: info logging [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\2382164240.py 12]


In [2]:
image_datapath = pathlib.Path(r"C:\Users\amrul\programming\datasets\various_images")
cifar10 = torchvision.datasets.CIFAR10(image_datapath, train = True, download = True, transform = transforms.ToTensor())

Files already downloaded and verified


In [3]:
cifar10_images = torch.stack([img for img,_ in cifar10],dim=-1)
print(f"cifar10_images.shape : {cifar10_images.shape}")

cifar10_images.shape : torch.Size([3, 32, 32, 50000])


In [6]:
cifar10_means = cifar10_images.view(3,-1).mean(dim=-1)
cifar10_stds = cifar10_images.view(3,-1).std(dim=-1)
print(f"cifar10 means : {cifar10_means}, cifar10 stds : {cifar10_stds}")

cifar10 means : tensor([0.4914, 0.4822, 0.4465]), cifar10 stds : tensor([0.2470, 0.2435, 0.2616])


In [7]:
cifar10_train_means = torch.tensor([0.4914, 0.4822, 0.4465])
cifar10_train_stds = torch.tensor([0.2470, 0.2435, 0.2616])

In [8]:
cifar10 = torchvision.datasets.CIFAR10(image_datapath, train = True, download = True, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(cifar10_train_means, cifar10_train_stds)]))
cifar10_val = torchvision.datasets.CIFAR10(image_datapath, train = False, download = True, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(cifar10_train_means, cifar10_train_stds)]))

Files already downloaded and verified
Files already downloaded and verified


In [9]:
cifar10_classes = cifar10.classes
print(f"cifar10 classes : {cifar10_classes}")

cifar10 classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [12]:
ba_label_map = {2:0,0:1}
ba_cifar = [(img, ba_label_map[label]) for img, label in cifar10 if label in ba_label_map.keys()]
ba_cifar_val = [(img, ba_label_map[label]) for img, label in cifar10_val if label in ba_label_map.keys()]

In [18]:
ba_cifar_val_images = torch.stack([img for img, _ in ba_cifar_val],dim=0)
ba_cifar_val_labels = torch.stack([torch.tensor([label]) for _,label in ba_cifar_val], dim=0)
print(f"ba_cifar_val_images : {ba_cifar_val_images.shape}, ba_cifar_val_labels.shape : {ba_cifar_val_labels.shape}")

ba_cifar_val_images : torch.Size([2000, 3, 32, 32]), ba_cifar_val_labels.shape : torch.Size([2000, 1])


In [19]:
import torch.nn as nn

In [20]:
model = nn.Sequential(nn.Linear(3*32*32,500),nn.Tanh(),nn.Linear(500,100),nn.Tanh(),nn.Linear(100,2),nn.LogSoftmax())
loss_fn = nn.NLLLoss()

In [26]:
val_pred = model(ba_cifar_val_images.view(2000,-1))
val_loss = loss_fn(val_pred,ba_cifar_val_labels.squeeze(-1))
print(f"val_pred.shape : {val_pred.shape}, val_loss : {val_loss}")

val_pred.shape : torch.Size([2000, 2]), val_loss : 0.7028641104698181


In [41]:
len(ba_cifar)

10000

In [61]:
def train_loop(model, loss_fn, optimizer, epochs):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        for idx, (img, label) in enumerate(ba_cifar):
            optimizer.zero_grad()
            x = img.unsqueeze(0).view(1,-1)

            y_pred = model(x)
            y_true = torch.tensor([label])
            loss = loss_fn(y_pred, y_true)            

            loss.backward()
            if idx%1000==0:
                logger.debug(f"epoch {epoch + 1} iteration {idx+1} , loss : {loss}")
                        
            optimizer.step()
        if epoch%10==0:                
            train_losses.append(loss)
            val_pred = model(ba_cifar_val_images.view(2000,-1))
            val_loss = loss_fn(val_pred,ba_cifar_val_labels.squeeze(-1))
            val_losses.append(val_loss)
            logger.debug(f"epoch {epoch + 1} loss : {loss}, val_loss : {val_loss}")
    return train_losses, val_losses
            

In [28]:
import torch.optim as optim

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [62]:
logging.debug("debug logging")
logging.info("info logging")

2022-07-16 18:36:25,888 DEBUG: debug logging [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\909002374.py 1]
2022-07-16 18:36:25,894 INFO: info logging [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\909002374.py 2]


In [ ]:
train_losses, val_losses = train_loop(model,loss_fn,optimizer,100)

# Let's use DataLoader

In [64]:
ba_cifar_loader = torch.utils.data.DataLoader(ba_cifar,batch_size=100, shuffle = True)
ba_cifar_val_loader = torch.utils.data.DataLoader(ba_cifar_val,batch_size=100, shuffle=True)

In [71]:
def calc_val_loss(val_data_loader):
    losses=[]
    for images, labels in val_data_loader:
        batch_size = images.shape[0]
        x = images.view(batch_size, -1)
        val_pred = model(x)
        loss = loss_fn(val_pred, labels)
        losses.append(loss)
    return torch.tensor(losses).mean()

In [73]:
val_loss = calc_val_loss(ba_cifar_val_loader)
logging.info(f"validation loss : {val_loss}")

2022-07-16 18:46:41,383 INFO: validation loss : 1.1048481464385986 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\760710048.py 2]


In [76]:
def train_loop(model, loss_fn, optimizer, epochs):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        for idx, (images, labels) in enumerate(ba_cifar_loader):
            optimizer.zero_grad()
            batch_size = images.shape[0]
            x = images.view(batch_size,-1)

            y_pred = model(x)
            y_true = labels
            loss = loss_fn(y_pred, y_true)            

            loss.backward()
            if idx%10==0:
                logger.debug(f"epoch {epoch + 1} iteration {idx+1} , loss : {loss}")
                        
            optimizer.step()
        if epoch%10==0:                
            train_losses.append(loss)            
            val_loss = calc_val_loss(ba_cifar_val_loader)
            val_losses.append(val_loss)
            logger.debug(f"epoch {epoch + 1} loss : {loss}, val_loss : {val_loss}")
    return train_losses, val_losses
            

In [77]:
train_losses, val_losses = train_loop(model,loss_fn,optimizer,100)

C:\Users\amrul\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
2022-07-16 18:48:24,713 DEBUG: epoch 1 iteration 1 , loss : 0.06534414738416672 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:24,939 DEBUG: epoch 1 iteration 11 , loss : 0.09751119464635849 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:25,147 DEBUG: epoch 1 iteration 21 , loss : 0.033627837896347046 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:25,381 DEBUG: epoch 1 iteration 31 , loss : 0.09217658638954163 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:25,576 DEBUG: epoch 1 iteration 41 , loss : 0.09018182009458542 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24

2022-07-16 18:48:36,162 DEBUG: epoch 6 iteration 11 , loss : 0.10270091891288757 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:36,390 DEBUG: epoch 6 iteration 21 , loss : 0.035586241632699966 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:36,619 DEBUG: epoch 6 iteration 31 , loss : 0.05200676992535591 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:36,877 DEBUG: epoch 6 iteration 41 , loss : 0.14623677730560303 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:37,101 DEBUG: epoch 6 iteration 51 , loss : 0.13491541147232056 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:37,309 DEBUG: epoch 6 iteration 61 , loss : 0.06613295525312424 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:37,544 DEBUG: epoch 6 iteration 71 , loss : 0.11317355185747147 

2022-07-16 18:48:48,205 DEBUG: epoch 11 iteration 51 , loss : 0.07278748601675034 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:48,414 DEBUG: epoch 11 iteration 61 , loss : 0.055215731263160706 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:48,636 DEBUG: epoch 11 iteration 71 , loss : 0.02786773443222046 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:48,848 DEBUG: epoch 11 iteration 81 , loss : 0.03648354858160019 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:49,066 DEBUG: epoch 11 iteration 91 , loss : 0.08258432149887085 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:48:49,424 DEBUG: epoch 11 loss : 0.06430206447839737, val_loss : 1.0916931629180908 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 23]
2022-07-16 18:48:49,445 DEBUG: epoch 12 iteration 1 , loss 

2022-07-16 18:48:59,987 DEBUG: epoch 16 iteration 81 , loss : 0.15862101316452026 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:00,220 DEBUG: epoch 16 iteration 91 , loss : 0.03008044883608818 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:00,440 DEBUG: epoch 17 iteration 1 , loss : 0.11039213836193085 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:00,655 DEBUG: epoch 17 iteration 11 , loss : 0.06409110873937607 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:00,898 DEBUG: epoch 17 iteration 21 , loss : 0.09422386437654495 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:01,123 DEBUG: epoch 17 iteration 31 , loss : 0.08571933954954147 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:01,339 DEBUG: epoch 17 iteration 41 , loss : 0.0955378264188

2022-07-16 18:49:11,689 DEBUG: epoch 22 iteration 11 , loss : 0.08228825777769089 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:11,925 DEBUG: epoch 22 iteration 21 , loss : 0.08916067332029343 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:12,165 DEBUG: epoch 22 iteration 31 , loss : 0.02608964405953884 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:12,404 DEBUG: epoch 22 iteration 41 , loss : 0.03717939928174019 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:12,639 DEBUG: epoch 22 iteration 51 , loss : 0.07147043198347092 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:12,876 DEBUG: epoch 22 iteration 61 , loss : 0.06621948629617691 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:49:13,123 DEBUG: epoch 22 iteration 71 , loss : 0.064570561051

...

2022-07-16 18:51:07,993 DEBUG: epoch 75 iteration 61 , loss : 0.054502278566360474 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:08,210 DEBUG: epoch 75 iteration 71 , loss : 0.09676450490951538 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:08,397 DEBUG: epoch 75 iteration 81 , loss : 0.04401557520031929 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:08,596 DEBUG: epoch 75 iteration 91 , loss : 0.039848681539297104 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:08,820 DEBUG: epoch 76 iteration 1 , loss : 0.013628089800477028 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:09,034 DEBUG: epoch 76 iteration 11 , loss : 0.04675404727458954 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:09,246 DEBUG: epoch 76 iteration 21 , loss : 0.0338805094

2022-07-16 18:51:19,716 DEBUG: epoch 81 iteration 1 , loss : 0.08150225877761841 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:19,908 DEBUG: epoch 81 iteration 11 , loss : 0.014627844095230103 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:20,132 DEBUG: epoch 81 iteration 21 , loss : 0.022850720211863518 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:20,333 DEBUG: epoch 81 iteration 31 , loss : 0.054754309356212616 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:20,537 DEBUG: epoch 81 iteration 41 , loss : 0.06532800942659378 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:20,751 DEBUG: epoch 81 iteration 51 , loss : 0.03412698209285736 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:20,966 DEBUG: epoch 81 iteration 61 , loss : 0.0499547272

2022-07-16 18:51:31,126 DEBUG: epoch 86 iteration 31 , loss : 0.0579352006316185 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:31,334 DEBUG: epoch 86 iteration 41 , loss : 0.03430330753326416 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:31,542 DEBUG: epoch 86 iteration 51 , loss : 0.018402332440018654 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:31,735 DEBUG: epoch 86 iteration 61 , loss : 0.06402851641178131 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:31,962 DEBUG: epoch 86 iteration 71 , loss : 0.02119435742497444 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:32,171 DEBUG: epoch 86 iteration 81 , loss : 0.027843192219734192 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:32,390 DEBUG: epoch 86 iteration 91 , loss : 0.03348765149

2022-07-16 18:51:42,656 DEBUG: epoch 91 iteration 71 , loss : 0.015349836088716984 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:42,865 DEBUG: epoch 91 iteration 81 , loss : 0.02543427236378193 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:43,075 DEBUG: epoch 91 iteration 91 , loss : 0.03568092733621597 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:43,414 DEBUG: epoch 91 loss : 0.04183724522590637, val_loss : 1.278930425643921 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 23]
2022-07-16 18:51:43,448 DEBUG: epoch 92 iteration 1 , loss : 0.02726166881620884 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:43,657 DEBUG: epoch 92 iteration 11 , loss : 0.08126276731491089 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:43,866 DEBUG: epoch 92 iteration 21 , loss :

2022-07-16 18:51:54,172 DEBUG: epoch 97 iteration 1 , loss : 0.0563087984919548 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:54,404 DEBUG: epoch 97 iteration 11 , loss : 0.11341550201177597 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:54,607 DEBUG: epoch 97 iteration 21 , loss : 0.02473311498761177 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:54,836 DEBUG: epoch 97 iteration 31 , loss : 0.03514251112937927 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:55,045 DEBUG: epoch 97 iteration 41 , loss : 0.04961102083325386 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:55,259 DEBUG: epoch 97 iteration 51 , loss : 0.03463074937462807 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\3627066196.py 16]
2022-07-16 18:51:55,470 DEBUG: epoch 97 iteration 61 , loss : 0.04275700449943

In [79]:
print(train_losses)
print(val_losses)

[tensor(0.0879, grad_fn=<NllLossBackward0>), tensor(0.0643, grad_fn=<NllLossBackward0>), tensor(0.1188, grad_fn=<NllLossBackward0>), tensor(0.0534, grad_fn=<NllLossBackward0>), tensor(0.0387, grad_fn=<NllLossBackward0>), tensor(0.0362, grad_fn=<NllLossBackward0>), tensor(0.0291, grad_fn=<NllLossBackward0>), tensor(0.0104, grad_fn=<NllLossBackward0>), tensor(0.0776, grad_fn=<NllLossBackward0>), tensor(0.0418, grad_fn=<NllLossBackward0>)]
[tensor(1.0907), tensor(1.0917), tensor(1.1182), tensor(1.1459), tensor(1.1725), tensor(1.1955), tensor(1.2154), tensor(1.2387), tensor(1.2589), tensor(1.2789)]


In [86]:
data_loader_generator = ((images,labels) for images, labels in ba_cifar_val_loader)
images,labels=next(data_loader_generator)

In [88]:
batch_size=images.shape[0]
x=images.view(batch_size,-1)
y=model(x)
y_max_vals,y_max_indices=torch.max(y,dim=-1)
logging.info(f"y_max_vals : {y_max_vals}")
logging.info(f"y_max_indices : {y_max_indices}")

C:\Users\amrul\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
2022-07-16 19:11:14,317 INFO: y_max_vals : tensor([-2.3842e-07, -2.6226e-06, -1.1895e-02, -1.7291e-01,  0.0000e+00,
        -7.5371e-04, -5.7598e-03, -7.3335e-02, -3.4571e-06,  0.0000e+00,
         0.0000e+00, -3.7301e-03, -3.9920e-01, -1.0011e-03,  0.0000e+00,
         0.0000e+00, -2.0742e-05, -9.5367e-07,  0.0000e+00, -4.3781e-02,
        -1.6673e-03, -2.1661e-01, -1.1921e-07, -1.8477e-05,  0.0000e+00,
        -8.8386e-03, -4.7684e-07, -1.9083e-03,  0.0000e+00, -1.8782e-03,
        -3.5763e-06, -3.2965e-01, -2.3241e-03, -7.9271e-05, -8.7545e-04,
        -1.1103e-01,  0.0000e+00, -5.6375e-01, -6.5198e-04, -2.9122e-01,
         0.0000e+00, -1.4782e-05, -5.0424e-05,  0.0000e+00,  0.0000e+00,
        -2.1517e-01, -4.5599e-04, -1.4896e-

In [87]:
logging.info(f"images shape : {images.shape}, label shape : {labels.shape}")

2022-07-16 19:08:37,338 INFO: images shape : torch.Size([100, 3, 32, 32]), label shape : torch.Size([100]) [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\1635776128.py 1]


In [89]:
def accuracy(data_loader):
    total_count=0
    correct_count=0
    for images, labels in data_loader:        
        batch_size = images.shape[0]
        x = images.view(batch_size, -1)
        total_count+=batch_size
        y = model(x)
        _,preds=torch.max(y,dim=-1)
        correct_count+=(preds==labels).sum()
        acc=correct_count/total_count
    return acc
        

In [90]:
train_acc = accuracy(ba_cifar_loader)
val_acc = accuracy(ba_cifar_val_loader)
logging.info(f"train accuracy : {train_acc}, val accuracy : {val_acc}")

C:\Users\amrul\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
2022-07-16 19:13:46,046 INFO: train accuracy : 0.9861000180244446, val accuracy : 0.7900000214576721 [in C:\Users\amrul\AppData\Local\Temp\ipykernel_24936\1565097608.py 3]


Let's count number of total parameters using parameters's ```numel``` method

In [93]:
import numpy as np
np.sum([p.numel() for p in model.parameters()])

1586802